# Scaling-Up Raster Data Analysis

*March 12, 2024*

```sh
pip install dask numpy scipy
```

[**Download the dataset here and it make it accessible to this notebook.**](http://files.ntsg.umt.edu/data/GIS_Programming/data/MOD17A3HGF_C61_h19v08_annual_NPP_gCm-2yr-1_2001-2021.h5)

In [ ]:
import h5py

hdf = h5py.File('data/MOD17A3HGF_C61_h19v08_annual_NPP_gCm-2yr-1_2001-2021.h5', 'r')

## Introduction

- A good chip circa 2004: [https://en.wikipedia.org/wiki/Pentium_4](https://en.wikipedia.org/wiki/Pentium_4)
- A good chip circa 2024: [https://ark.intel.com/content/www/us/en/ark/products/236781/intel-core-i7-processor-14700-33m-cache-up-to-5-40-ghz.html](https://ark.intel.com/content/www/us/en/ark/products/236781/intel-core-i7-processor-14700-33m-cache-up-to-5-40-ghz.html)

## Bottlenecks in Computing

![](Moore.png)

*Image courtesy of XSEDE*

[Quantum tunneling is one of the reasons it's hard to make smaller and smaller transistors.](https://en.wikipedia.org/wiki/Quantum_tunnelling#Electronics)

---

## Motivation

Before we start working with huge raster datasets, let's explore some of the basic tools involved in parallel processing.

In [ ]:
def factorial(n):
    result = 1
    for i in range(n, 1, -1):
        result = result * i
    return result

In [ ]:
from matplotlib import pyplot
import timeit

times = []
for n in my_numbers:
    t = timeit.timeit(f"factorial({n})", globals = globals(), number = 1000)
    times.append(t)
    
pyplot.bar(my_numbers, times)
pyplot.xlabel('Number of Values')
pyplot.ylabel('Execution Time (secs)')

---

## Concurrent Processes

---

## Bottlenecks in Array Processing

---

## Analyzing Large Raster Data Cubes

In [ ]:
import h5py

hdf = h5py.File('data/MOD17A3HGF_C61_h19v08_annual_NPP_gCm-2yr-1_2001-2021.h5', 'r')
npp = hdf['NPP'][:]
hdf.close()

npp.shape

In [ ]:
from scipy import stats

def linear_trend(array, n_years = 21):
    # linregress(x, y) takes two arguments: y is regressed on x
    result = stats.linregress(np.arange(0, n_years), array)
    return result[0] # Just the slope

### Estimating Time to Completion

---

## Concurrency in Python

![](concurrency.jpg)

*Image by Kelvin Wahome*

### Independent Processes

### How Many Processes?

In [ ]:
import os

os.cpu_count()

In [ ]:
import psutil 

psutil.cpu_count(logical = False)

---

## Concurrent Raster Data Processing

**Now, let's see how to actually do concurrent raster processing with `dask`.**

When doing concurrent operations on arrays, we want to let `dask` do most of the work figuring out exactly how to chunk up the spatial domain.

In [ ]:
def maxval(x):
    return np.array([np.max(x[:,i]) for i in range(0, x[0].size)])

In [ ]:
d_npp = da.from_array(npp).reshape((21, 2400*2400)).rechunk((-1, 'auto'), block_size_limit = 65e6)
d_npp

In [ ]:
d_npp = da.from_array(npp).reshape((21, 2400*2400)).rechunk((-1, 1440000))
d_npp

In [ ]:
from matplotlib import pyplot

max_npp = max_npp.reshape((2400,2400)).astype(np.float32)
max_npp[max_npp < 0] = np.nan

pyplot.imshow(max_npp, interpolation = 'nearest')
pyplot.colorbar()
pyplot.show()

---

## At Home: Calculate a Linear Trend

Try running `map_blocks()` with a different function! The `linear_trend2()` function, below, is designed to calculate linear trends on each pixel in a block of pixels, so it can be used with `dask` and `map_blocks()` to calculate trends over large raster data cubes.

This took about 10 minutes with 4 processes on my Intel i7 CPU at home. You should expect it to take about the same amount of time.

In [ ]:
def linear_trend2(block, n_years = 21):
    x_data = np.arange(0, n_years)
    # A fancy way of saying call stats.linregress() in a for loop,
    #    once for each pixel in the block
    return np.array([
        stats.linregress(x_data, block[:,i])[0]
        for i in range(0, block[0].size)
    ])

In [56]:
%%timeit -n 1 -r 1

result = d_npp.map_blocks(linear_trend2, drop_axis = 0, dtype = np.int16)
result.compute()

10min 4s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)
